# Inference - Classification, BBox, Angle

## Parameters

In [2]:
# test 0: Full unit circle, test 1: Right half plane, test 2: Regular angle
test = 0
img_list_file = 'list_of_img_in_val_set_18-03.csv'
place_to_store_results = 'Evaluations/something_else/'
path_img_folder = '../../03 Data/Simple Dataset/'
accept_BB_threshold = 0.9

model_folder = "./snapshots/"
model_name = "Full_Unit_Circle_SmoothL1_resnet50_csv_25.h5"

## Imports

In [3]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from numpy import genfromtxt
import time
import json
import copy

# use this to change which GPU to use
gpu = 0
# set the modified tf session as backend in keras  
#setup_gpu(gpu)  #NOTICE: enable when using paperspace server!!

Using TensorFlow backend.


## Functions

In [7]:
def convert_rect_to_point(rect):
    x = rect[0]
    y = rect[1]
    w = rect[2]
    h = rect[3]
    x1 = x
    y1 = y
    x2 = x + w
    y2 = y + h
    
    return (x1, y1, x2, y2)

def read_annotations_from_json(img_name):
    with open(path_img_folder + img_name.strip('.png') + '.json') as json_file:
        data = json.load(json_file)
        shapes_list = data['shapes']
        bbox_list = []
        angle_list = []
        for annotation in shapes_list: 
            rect = cv2.boundingRect(np.float32(annotation['points']))
            rot_rect = cv2.minAreaRect(np.float32(annotation['points']))
            if rot_rect[-2][0] < rot_rect[-2][1]:
                rot_rect = (rot_rect[0],(rot_rect[-2][1],rot_rect[-2][0]),rot_rect[-1]+90)
            # make sure that no angle is above 90 or below -90
            if rot_rect[-1] > 90:
                rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]-180)
            if rot_rect[-1] < -90:
                rot_rect = (rot_rect[0],rot_rect[1],rot_rect[-1]+180)
            angle_list.append(rot_rect[-1]/180*np.pi)
            bbox_list.append(convert_rect_to_point(rect))
    return bbox_list, angle_list; 


def max_val(val1, val2):
    if val1 > val2:
        return val1
    else:
        return val2
    
def min_val(val1, val2):
    if val1 > val2:
        return val2
    else:
        return val1
    
    
def filter_bounding_boxes(bounding_boxes, scores, angles):
    list_of_BB = []
    list_of_angles = []
    for box, score, angle in zip(bounding_boxes, scores, angles):
        if score < accept_BB_threshold: 
            break
        list_of_angles.append(angle)
        list_of_BB.append(box)
    return list_of_BB, list_of_angles

def format_angle(angles, test):
    formatted_angles = []
    if test == 0:
        for angle in angles:
            formatted_angles.append(np.arctan2(angle[1],angle[0])/2)
    elif test == 1:
        for angle in angles:
            formatted_angles.append(np.arctan2(angle[1],angle[0]))
    elif test == 2:
        formatted_angles.append(angle)
    return formatted_angles       

def area(BB):
    width = abs(BB[0] - BB[2])
    height = abs(BB[1] - BB[3])
    return height * width

def intersection(BB1, BB2):
    # find coordiantes of intersection rectangle 
    (x1, y1, x2, y2) = BB1
    (x3, y3, x4, y4) = BB2
    if x1 > x4 or x3 > x2 or y3 > y2 or y1 > y4:
        return 0
    x5 = max_val(x1, x3);
    y5 = max_val(y1, y3);
    x6 = min_val(x2, x4);
    y6 = min_val(y2, y4);  
    BB = (x5, y5, x6, y6)
    # TODO: check if no intersection exists
    return area(BB)
    
def union(BB1, BB2): 
    return area(BB1) + area(BB2) - intersection(BB1, BB2)

def evaluate_predictions(bbox_annotations, angle_annotations, bbox_preds, angle_preds, IOU_thres):
    bbox_predictions = copy.deepcopy(bbox_preds)
    angle_predictions = copy.deepcopy(angle_preds)
    false_neg = 0
    error_list = []
    for idx, annotation in enumerate(bbox_annotations): 
        #annotation = convert_rect_to_point(anno)
        max_IOU = 0
        for i in range(len(bbox_predictions)): 
            #print('Annotation: ', annotation)
            #print('pred candidate: ', predictions[i])
            if intersection(annotation, bbox_predictions[i]) > 0: #convert_rect_to_point only for tests
                IOU = intersection(annotation, bbox_predictions[i])/union(annotation, bbox_predictions[i])
                #print(IOU)  
                if(IOU > max_IOU):
                    max_IOU_index = i
                    max_IOU = IOU
        if max_IOU > IOU_thres:
            # remove prediction from list
            bbox_predictions.pop(max_IOU_index)
            error_list.append(angle_predictions[max_IOU_index] - angle_annotations[idx])
            angle_predictions.pop(max_IOU_index)
        else:
            false_neg = false_neg + 1
    
    false_pos = len(predictions)
    true_pos = len(bbox_preds) - len(predictions)
    return (false_neg, false_pos, true_pos), error_list 

def draw_vector(img, x, y, box):
    length = np.linalg.norm([x,y])*25
    angle = np.arctan2(y,x)
    angle = angle/2
    center_x = (box[0]+box[2])/2
    center_y = (box[1]+box[3])/2
    pt1_x = -length*np.cos(angle)+center_x
    pt1_y = -length*np.sin(angle)+center_y
    pt2_x = length*np.cos(angle)+center_x
    pt2_y = length*np.sin(angle)+center_y
    cv2.line(img, (int(pt1_x),int(pt1_y)), (int(pt2_x), int(pt2_y)), color = (0, 255,255), thickness = 2, lineType = 2)
    return

## Setup Model

In [5]:
# import model
model_path = os.path.join(model_folder, model_name);
model = models.load_model(model_path, backbone_name='resnet50');

# If model is not converted to inference model, use line below: 
model = models.convert_model(model);

# Mapping of model output and classes
labels_to_names = {0: 'Brick'};

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

## Predictions

In [8]:
# import data
file = open(img_list_file)
file_paths = list(file)  

bbox_prediction_list = [] 
angle_prediction_list = []

bbox_annotation_list = []
angle_annotation_list = []

alpha = 0.5

for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    # create copy to draw on 
    draw = image.copy()
    draw2 = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLhttp://cocodataset.org/#detection-evalOR_BGR2RGB)
    
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image 
    boxes, scores, labels, angles = model.predict_on_batch(np.expand_dims(image, axis=0))
    
    # correct for image scale
    boxes /= scale
    
    
    
    # print detections
    for box, score, label, angle in zip(boxes[0], scores[0], labels[0], angles[0]):
        # scores are sorted so we can break
        if score < accept_BB_threshold:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
        draw_vector(draw, angle[0], angle[1], box)
        draw_caption(draw2, b, caption)
        draw_vector(draw2, angle[0], angle[1], box)
    
    cv2.addWeighted(draw, alpha, draw2, 1 - alpha,0, draw)
    # save image 
    #cv2.imwrite(place_to_store_results + path.strip('\n'), draw)
    
    # load json file containing annotations
    bbox, angle = read_annotations_from_json(path.strip('\n'))
    bbox_annotation_list.append(bbox)
    angle_annotation_list.append(angle)
    
    # remove bounding boxes with a classification score below accept_BB_threshold. 
    print(boxes[0], scores[0], angles[0])
    bbox, angle = filter_bounding_boxes(boxes[0], scores[0], angles[0])
    bbox_prediction_list.append(bbox)
    angle_prediction_list.append(angle)
    
    angle_prediction_list = format_angle(angle_prediction_list, test)
    
print(annotation_list)
print(prediction_list)

[[426.8996  202.1844  531.68494 232.33188]
 [321.8541  325.8536  425.46414 354.92334]
 [422.41895 333.43698 527.7315  362.04395]
 ...
 [ -0.6      -0.6      -0.6      -0.6    ]
 [ -0.6      -0.6      -0.6      -0.6    ]
 [ -0.6      -0.6      -0.6      -0.6    ]] [ 0.99994004  0.99992955  0.9999229   0.9999181   0.9999181   0.99989307
  0.9998559   0.9998418   0.99984014  0.9998123   0.9998104   0.9997861
  0.99967587  0.9996532   0.9996226   0.99921584  0.7718313   0.6678915
  0.06618837  0.06347981  0.05775914  0.05735795  0.05615484  0.05454683
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.    

[[423.02774 250.49503 527.8926  280.01862]
 [425.72037 202.20837 529.34406 230.75636]
 [320.89124 298.76773 427.57938 327.27933]
 ...
 [ -0.6      -0.6      -0.6      -0.6    ]
 [ -0.6      -0.6      -0.6      -0.6    ]
 [ -0.6      -0.6      -0.6      -0.6    ]] [ 0.99990535  0.99987996  0.999877    0.9998479   0.9998049   0.99978405
  0.99974257  0.9996551   0.9996244   0.99950624  0.99949694  0.99935097
  0.99929893  0.9992416   0.9989471   0.9988746   0.79155004  0.5554113
  0.08637314  0.06795128  0.05493585 -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1

[[320.9674  322.52438 425.4062  356.6013 ]
 [328.1458  195.32404 430.3137  227.18077]
 [428.15698 227.31587 531.38086 259.66727]
 ...
 [ -0.6      -0.6      -0.6      -0.6    ]
 [ -0.6      -0.6      -0.6      -0.6    ]
 [ -0.6      -0.6      -0.6      -0.6    ]] [ 0.9999691   0.99994004  0.9999391   0.99992585  0.99992037  0.99991107
  0.9998919   0.9998779   0.99987674  0.99987185  0.9998485   0.99980086
  0.99977046  0.9996469   0.99961984  0.99946743  0.08331561  0.08273647
  0.0575739   0.05471651  0.053979    0.05139212 -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -

IndexError: invalid index to scalar variable.

## Evaluate

In [ ]:
IOU_thres = np.arange(0.5, 1, 0.05).tolist()

precision_list = []
recall_list = []
f1_list = []

for thres in IOU_thres: 
    print('Threshold: ', thres)
    total_false_positive = 0
    total_true_positive = 0
    total_false_negative = 0
    for i in range(len(bbox_annotation_list)):
        #calculate intersection and union of two bounding boxes
        (false_neg, false_pos, true_pos), angle_error = evaluate_predictions(bbox_annotation_list[i], angle_annotation_list[i], 
                                                                             bbox_prediction_list[i], angle_prediction_list[i], thres)
        #results = evaluate_predictions(annotations, annotations)
        total_false_negative = total_false_negative + false_neg
        total_false_positive = total_false_positive + false_pos
        total_true_positive  = total_true_positive  + true_pos
        
    # Calculate precision, recall and F1 score 
    precision = total_true_positive / (total_true_positive + total_false_positive)
    precision_list.append(precision)
    recall = total_true_positive / (total_true_positive + total_false_negative)
    recall_list.append(recall)
    f1 = 2 * (precision * recall) / (precision + recall) 
    f1_list.append(f1)
    
    # print result
    print(angle_error)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('f1: ', f1)
    
print(precision_list)
print(recall_list)
print(f1_list)

## Plot

In [ ]:
plt.figure(figsize=(4, 3), dpi=200)
plt.plot(IOU_thres, precision_list)
plt.plot(IOU_thres, recall_list)
plt.plot(IOU_thres, f1_list)
plt.legend(['Precision', 'Recall', 'F1'])
plt.title('AP score, RetinaNet, Validation set')
plt.ylabel('Score')
plt.xlabel('IOU Threshold')


average_precision = np.average(precision_list)
average_recall = np.average(recall_list)
average_f1 = np.average(f1_list)

print('Avg. precision:', average_precision)
print('Avg. recall:   ', average_recall)
print('Avg. F1:       ', average_f1)